In [1]:
#!git clone https://github.com/fchollet/keras.git && cd keras && python setup.py install --user

In [2]:
from os import listdir
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential, Model
from keras.utils import to_categorical
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers import Embedding, TimeDistributed, RepeatVector, LSTM, concatenate , Input, Reshape
from keras.preprocessing.image import array_to_img, img_to_array, load_img
from keras.applications.inception_resnet_v2 import InceptionResNetV2, preprocess_input
from keras.callbacks import ModelCheckpoint
import numpy as np
from numpy import argmax
import os

Using TensorFlow backend.


In [3]:
images = []
all_filenames = os.listdir('images/')
all_filenames.sort()
for filename in all_filenames:
    print(filename)
    images.append(img_to_array(load_img('images/'+filename, target_size=(299, 299))))
images = np.array(images, dtype=float)
images = preprocess_input(images)

IR2 = InceptionResNetV2(weights=None, include_top=False, pooling='avg')
IR2.load_weights('/data/models/inception_resnet_v2_weights_tf_dim_ordering_tf_kernels_notop.h5')
features = IR2.predict(images)

86.jpg
87.jpg
88.jpg
89.jpg
90.jpg


In [4]:
max_caption_len = 100
tokenizer = Tokenizer(filters='', split=" ", lower=False)

def load_doc(filename):
    file = open(filename, 'r')
    text = file.read()
    file.close()
    return text

X = []
all_filenames = os.listdir('html/')
all_filenames.sort()
for filename in all_filenames:
    X.append(load_doc('html/'+filename))

tokenizer.fit_on_texts(X)

vocab_size = len(tokenizer.word_index) + 1
sequences = tokenizer.texts_to_sequences(X)
max_length = max(len(s) for s in sequences)
 
X, y, image_data = list(), list(), list()
for img_no, seq in enumerate(sequences):
    for i in range(1, len(seq)):
        in_seq, out_seq = seq[:i], seq[i]
        in_seq = pad_sequences([in_seq], maxlen=max_length)[0]
        out_seq = to_categorical([out_seq], num_classes=vocab_size)[0]
        image_data.append(features[img_no])
        X.append(in_seq[-100:])
        y.append(out_seq)

X, y, image_data = np.array(X), np.array(y), np.array(image_data)

In [5]:
print(image_data.shape)
print(X.shape)
print(y.shape)
print(features.shape)

(2306, 1536)
(2306, 100)
(2306, 436)
(5, 1536)


In [6]:
image_features = Input(shape=(1536,))
image_flat = Dense(128, activation='relu')(image_features)
ir2_out = RepeatVector(max_caption_len)(image_flat)

language_input = Input(shape=(max_caption_len,))
language_model = Embedding(vocab_size, 200, input_length=max_caption_len)(language_input)
language_model = LSTM(256, return_sequences=True)(language_model)
language_model = LSTM(256, return_sequences=True)(language_model)
language_model = TimeDistributed(Dense(128, activation='relu'))(language_model)

decoder = concatenate([ir2_out, language_model])
decoder = LSTM(512, return_sequences=True)(decoder)
decoder = LSTM(512, return_sequences=False)(decoder)
decoder_output = Dense(vocab_size, activation='softmax')(decoder)

model = Model(inputs=[image_features, language_input], outputs=decoder_output)
model.compile(loss='categorical_crossentropy', optimizer='rmsprop')

In [7]:
#Train model
filepath="org-weights-epoch-{epoch:04d}---loss-{loss:.4f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_weights_only=True, period=50)
callbacks_list = [checkpoint]

In [8]:
model.fit([image_data, X], y, batch_size=1000, callbacks=callbacks_list, shuffle=False, epochs=1000)

Epoch 1/1000
2306/2306 [==============================] - 10s 4ms/step - loss: 5.9856
Epoch 2/1000
2306/2306 [==============================] - 7s 3ms/step - loss: 5.8739
Epoch 3/1000
2306/2306 [==============================] - 7s 3ms/step - loss: 5.7184
Epoch 4/1000
2306/2306 [==============================] - 7s 3ms/step - loss: 5.6836
Epoch 5/1000
2306/2306 [==============================] - 7s 3ms/step - loss: 5.6753
Epoch 6/1000
2306/2306 [==============================] - 7s 3ms/step - loss: 5.6701
Epoch 7/1000
2306/2306 [==============================] - 7s 3ms/step - loss: 5.6529
Epoch 8/1000
2306/2306 [==============================] - 7s 3ms/step - loss: 5.6137
Epoch 9/1000
2306/2306 [==============================] - 7s 3ms/step - loss: 5.5642
Epoch 10/1000
2306/2306 [==============================] - 7s 3ms/step - loss: 5.5217
Epoch 11/1000
2306/2306 [==============================] - 7s 3ms/step - loss: 5.5384
Epoch 12/1000
2306/2306 [==============================] - 7s 

2306/2306 [==============================] - 7s 3ms/step - loss: 4.7702
Epoch 95/1000
2306/2306 [==============================] - 7s 3ms/step - loss: 4.5742
Epoch 96/1000
2306/2306 [==============================] - 7s 3ms/step - loss: 4.4247
Epoch 97/1000
2306/2306 [==============================] - 7s 3ms/step - loss: 4.2660
Epoch 98/1000
2306/2306 [==============================] - 7s 3ms/step - loss: 4.3654
Epoch 99/1000
2306/2306 [==============================] - 7s 3ms/step - loss: 4.3094
Epoch 100/1000
2306/2306 [==============================] - 7s 3ms/step - loss: 4.3041
Epoch 101/1000
2306/2306 [==============================] - 7s 3ms/step - loss: 4.1955
Epoch 102/1000
2306/2306 [==============================] - 7s 3ms/step - loss: 4.2334
Epoch 103/1000
2306/2306 [==============================] - 7s 3ms/step - loss: 4.2633
Epoch 104/1000
2306/2306 [==============================] - 7s 3ms/step - loss: 4.0798
Epoch 105/1000
2306/2306 [==============================] - 7s 

2306/2306 [==============================] - 7s 3ms/step - loss: 2.4148
Epoch 186/1000
2306/2306 [==============================] - 7s 3ms/step - loss: 2.3467
Epoch 187/1000
2306/2306 [==============================] - 7s 3ms/step - loss: 2.3633
Epoch 188/1000
2306/2306 [==============================] - 7s 3ms/step - loss: 2.4475
Epoch 189/1000
2306/2306 [==============================] - 7s 3ms/step - loss: 2.5979
Epoch 190/1000
2306/2306 [==============================] - 7s 3ms/step - loss: 2.4428
Epoch 191/1000
2306/2306 [==============================] - 7s 3ms/step - loss: 2.2331
Epoch 192/1000
2306/2306 [==============================] - 7s 3ms/step - loss: 3.5901
Epoch 193/1000
2306/2306 [==============================] - 7s 3ms/step - loss: 2.7582
Epoch 194/1000
2306/2306 [==============================] - 7s 3ms/step - loss: 2.8715
Epoch 195/1000
2306/2306 [==============================] - 7s 3ms/step - loss: 2.5422
Epoch 196/1000
2306/2306 [==============================] 

2306/2306 [==============================] - 7s 3ms/step - loss: 2.1140
Epoch 277/1000
2306/2306 [==============================] - 7s 3ms/step - loss: 1.6548
Epoch 278/1000
2306/2306 [==============================] - 7s 3ms/step - loss: 1.6173
Epoch 279/1000
2306/2306 [==============================] - 7s 3ms/step - loss: 1.7175
Epoch 280/1000
2306/2306 [==============================] - 7s 3ms/step - loss: 1.8761
Epoch 281/1000
2306/2306 [==============================] - 7s 3ms/step - loss: 1.9094
Epoch 282/1000
2306/2306 [==============================] - 7s 3ms/step - loss: 1.8937
Epoch 283/1000
2306/2306 [==============================] - 7s 3ms/step - loss: 1.7004
Epoch 284/1000
2306/2306 [==============================] - 7s 3ms/step - loss: 1.6701
Epoch 285/1000
2306/2306 [==============================] - 7s 3ms/step - loss: 1.8255
Epoch 286/1000
2306/2306 [==============================] - 7s 3ms/step - loss: 1.9333
Epoch 287/1000
2306/2306 [==============================] 

2306/2306 [==============================] - 7s 3ms/step - loss: 0.2239
Epoch 368/1000
2306/2306 [==============================] - 7s 3ms/step - loss: 0.2517
Epoch 369/1000
2306/2306 [==============================] - 7s 3ms/step - loss: 0.2421
Epoch 370/1000
2306/2306 [==============================] - 7s 3ms/step - loss: 0.4156
Epoch 371/1000
2306/2306 [==============================] - 7s 3ms/step - loss: 0.7061
Epoch 372/1000
2306/2306 [==============================] - 7s 3ms/step - loss: 0.3613
Epoch 373/1000
2306/2306 [==============================] - 7s 3ms/step - loss: 0.2066
Epoch 374/1000
2306/2306 [==============================] - 7s 3ms/step - loss: 0.1947
Epoch 375/1000
2306/2306 [==============================] - 7s 3ms/step - loss: 0.2076
Epoch 376/1000
2306/2306 [==============================] - 7s 3ms/step - loss: 0.9291
Epoch 377/1000
2306/2306 [==============================] - 7s 3ms/step - loss: 0.2593
Epoch 378/1000
2306/2306 [==============================] 

...

2306/2306 [==============================] - 7s 3ms/step - loss: 2.3083e-04
Epoch 639/1000
2306/2306 [==============================] - 7s 3ms/step - loss: 2.1337e-04
Epoch 640/1000
2306/2306 [==============================] - 7s 3ms/step - loss: 1.9791e-04
Epoch 641/1000
2306/2306 [==============================] - 7s 3ms/step - loss: 1.8373e-04
Epoch 642/1000
2306/2306 [==============================] - 7s 3ms/step - loss: 1.6791e-04
Epoch 643/1000
2306/2306 [==============================] - 7s 3ms/step - loss: 1.5591e-04
Epoch 644/1000
2306/2306 [==============================] - 7s 3ms/step - loss: 1.4369e-04
Epoch 645/1000
2306/2306 [==============================] - 7s 3ms/step - loss: 1.3386e-04
Epoch 646/1000
2306/2306 [==============================] - 7s 3ms/step - loss: 1.2606e-04
Epoch 647/1000
2306/2306 [==============================] - 7s 3ms/step - loss: 1.3092e-04
Epoch 648/1000
2306/2306 [==============================] - 7s 3ms/step - loss: 0.0047
Epoch 649/1000
230

2306/2306 [==============================] - 7s 3ms/step - loss: 1.3738e-04
Epoch 727/1000
2306/2306 [==============================] - 7s 3ms/step - loss: 1.2616e-04
Epoch 728/1000
2306/2306 [==============================] - 7s 3ms/step - loss: 1.1572e-04
Epoch 729/1000
2306/2306 [==============================] - 7s 3ms/step - loss: 1.0649e-04
Epoch 730/1000
2306/2306 [==============================] - 7s 3ms/step - loss: 9.8104e-05
Epoch 731/1000
2306/2306 [==============================] - 7s 3ms/step - loss: 9.0566e-05
Epoch 732/1000
2306/2306 [==============================] - 7s 3ms/step - loss: 8.3441e-05
Epoch 733/1000
2306/2306 [==============================] - 7s 3ms/step - loss: 7.6916e-05
Epoch 734/1000
2306/2306 [==============================] - 7s 3ms/step - loss: 7.0808e-05
Epoch 735/1000
2306/2306 [==============================] - 7s 3ms/step - loss: 6.5313e-05
Epoch 736/1000
2306/2306 [==============================] - 7s 3ms/step - loss: 6.0370e-05
Epoch 737/1000

2306/2306 [==============================] - 7s 3ms/step - loss: 1.5749e-05
Epoch 814/1000
2306/2306 [==============================] - 7s 3ms/step - loss: 1.4752e-05
Epoch 815/1000
2306/2306 [==============================] - 7s 3ms/step - loss: 1.3884e-05
Epoch 816/1000
2306/2306 [==============================] - 7s 3ms/step - loss: 1.3070e-05
Epoch 817/1000
2306/2306 [==============================] - 7s 3ms/step - loss: 1.2333e-05
Epoch 818/1000
2306/2306 [==============================] - 7s 3ms/step - loss: 1.1570e-05
Epoch 819/1000
2306/2306 [==============================] - 7s 3ms/step - loss: 1.0911e-05
Epoch 820/1000
2306/2306 [==============================] - 7s 3ms/step - loss: 1.0338e-05
Epoch 821/1000
2306/2306 [==============================] - 7s 3ms/step - loss: 9.7956e-06
Epoch 822/1000
2306/2306 [==============================] - 7s 3ms/step - loss: 9.3325e-06
Epoch 823/1000
2306/2306 [==============================] - 7s 3ms/step - loss: 8.9353e-06
Epoch 824/1000

Epoch 901/1000
2306/2306 [==============================] - 7s 3ms/step - loss: 1.9845e-06
Epoch 902/1000
2306/2306 [==============================] - 7s 3ms/step - loss: 1.9679e-06
Epoch 903/1000
2306/2306 [==============================] - 7s 3ms/step - loss: 1.9526e-06
Epoch 904/1000
2306/2306 [==============================] - 7s 3ms/step - loss: 1.9352e-06
Epoch 905/1000
2306/2306 [==============================] - 7s 3ms/step - loss: 1.9189e-06
Epoch 906/1000
2306/2306 [==============================] - 7s 3ms/step - loss: 1.9030e-06
Epoch 907/1000
2306/2306 [==============================] - 7s 3ms/step - loss: 1.8876e-06
Epoch 908/1000
2306/2306 [==============================] - 7s 3ms/step - loss: 1.8725e-06
Epoch 909/1000
2306/2306 [==============================] - 7s 3ms/step - loss: 1.8576e-06
Epoch 910/1000
2306/2306 [==============================] - 7s 3ms/step - loss: 1.8430e-06
Epoch 911/1000
2306/2306 [==============================] - 7s 3ms/step - loss: 1.8293e-06

2306/2306 [==============================] - 7s 3ms/step - loss: 1.1584e-06
Epoch 990/1000
2306/2306 [==============================] - 7s 3ms/step - loss: 1.1535e-06
Epoch 991/1000
2306/2306 [==============================] - 7s 3ms/step - loss: 1.1487e-06
Epoch 992/1000
2306/2306 [==============================] - 7s 3ms/step - loss: 1.1433e-06
Epoch 993/1000
2306/2306 [==============================] - 7s 3ms/step - loss: 1.1384e-06
Epoch 994/1000
2306/2306 [==============================] - 7s 3ms/step - loss: 1.1334e-06
Epoch 995/1000
2306/2306 [==============================] - 7s 3ms/step - loss: 1.1288e-06
Epoch 996/1000
2306/2306 [==============================] - 7s 3ms/step - loss: 1.1226e-06
Epoch 997/1000
2306/2306 [==============================] - 7s 3ms/step - loss: 1.1181e-06
Epoch 998/1000
2306/2306 [==============================] - 7s 3ms/step - loss: 1.1126e-06
Epoch 999/1000
2306/2306 [==============================] - 7s 3ms/step - loss: 1.1079e-06
Epoch 1000/100

In [9]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

<IPython.core.display.Javascript object>

In [10]:
# map an integer to a word
def word_for_id(integer, tokenizer):
    for word, index in tokenizer.word_index.items():
        if index == integer:
            return word
    return None

In [11]:
# generate a description for an image
def generate_desc(model, tokenizer, photo, max_length):
    # seed the generation process
    in_text = 'START'
    # iterate over the whole length of the sequence
    for i in range(900):
        # integer encode input sequence
        sequence = tokenizer.texts_to_sequences([in_text])[0][-100:]
        # pad input
        sequence = pad_sequences([sequence], maxlen=max_length)
        # predict next word
        yhat = model.predict([photo,sequence], verbose=0)
        # convert probability to integer
        yhat = argmax(yhat)
        # map integer to word
        word = word_for_id(yhat, tokenizer)
        # stop if we cannot map the word
        if word is None:
            break
        # append as input for generating the next word
        in_text += ' ' + word
        print(' ' + word, end='')
        # stop if we predict the end of the sequence
        if word == 'END':
            break
    return 

In [19]:
test_image = img_to_array(load_img('images/88.jpg', target_size=(299, 299)))
test_image = np.array(test_image, dtype=float)
test_image = preprocess_input(test_image)
test_features = IR2.predict(np.array([test_image]))
generate_desc(model, tokenizer, np.array(test_features), 100)

 <!DOCTYPE html>
<html lang="en" dir="ltr">
<head>
<title>Basic 88</title>
<meta charset="iso-8859-1">
<link rel="stylesheet" href="styles/layout.css" type="text/css">
<!--[if lt IE 9]><script src="scripts/html5shiv.js"></script><![endif]-->
</head>
<body>
<div class="wrapper row1">
 <header id="header" class="clear">
 <div id="hgroup">
 <h1><a href="#">Basic 88</a></h1>
 <h2>Free HTML5 Website Template</h2>
 </div>
 <nav>
 <ul>
 <li><a href="#">Text Link</a></li>
 <li><a href="#">Text Link</a></li>
 <li><a href="#">Text Link</a></li>
 <li><a href="#">Text Link</a></li>
 <li class="last"><a href="#">Text Link</a></li>
 </ul>
 </nav>
 </header>
</div>
<!-- content -->
<div class="wrapper row2">
 <div id="container" class="clear">
 <!-- Slider -->
 <section id="slider"><a href="#"><img src="images/demo/960x360.gif" alt=""></a></section>
 <!-- main content -->
 <div id="homepage">
 <!-- Services -->
 <section id="services" class="clear">
 <article class="one_third">
 <figure><img src="ima

In [20]:
test_image = img_to_array(load_img('images/89.jpg', target_size=(299, 299)))
test_image = np.array(test_image, dtype=float)
test_image = preprocess_input(test_image)
test_features = IR2.predict(np.array([test_image]))
generate_desc(model, tokenizer, np.array(test_features), 100)

 <!DOCTYPE html>
<html lang="en" dir="ltr">
<head>
<title>Basic 89</title>
<meta charset="iso-8859-1">
<link rel="stylesheet" href="styles/layout.css" type="text/css">
<!--[if lt IE 9]><script src="scripts/html5shiv.js"></script><![endif]-->
</head>
<body>
<div class="wrapper row1">
 <header id="header" class="clear">
 <div id="hgroup">
 <h1><a href="#">Basic 89</a></h1>
 <h2>Free HTML5 Website Template</h2>
 </div>
 <nav>
 <ul>
 <li><a href="#">Text Link</a></li>
 <li><a href="#">Text Link</a></li>
 <li><a href="#">Text Link</a></li>
 <li><a href="#">Text Link</a></li>
 <li class="last"><a href="#">Text Link</a></li>
 </ul>
 </nav>
 </header>
</div>
<!-- content -->
<div class="wrapper row2">
 <div id="container" class="clear">
 <!-- Slider -->
 <section id="slider"><a href="#"><img src="images/demo/960x360.gif" alt=""></a></section>
 <!-- main content -->
 <div id="homepage">
 <!-- services area -->
 <section id="services" class="clear">
 <article class="one_quarter">
 <figure><img s

In [21]:
test_image = img_to_array(load_img('images/90.jpg', target_size=(299, 299)))
test_image = np.array(test_image, dtype=float)
test_image = preprocess_input(test_image)
test_features = IR2.predict(np.array([test_image]))
generate_desc(model, tokenizer, np.array(test_features), 100)

 <!DOCTYPE html>
<html lang="en" dir="ltr">
<head>
<title>Basic 90</title>
<meta charset="iso-8859-1">
<link rel="stylesheet" href="styles/layout.css" type="text/css">
<!--[if lt IE 9]><script src="scripts/html5shiv.js"></script><![endif]-->
</head>
<body>
<div class="wrapper row1">
 <header id="header" class="clear">
 <div id="hgroup">
 <h1><a href="#">Basic 90</a></h1>
 <h2>Free HTML5 Website Template</h2>
 </div>
 <form action="#" method="post">
 <fieldset>
 <legend>Search:</legend>
 <input type="text" value="Search Our Website&hellip;" onFocus="this.value=(this.value=='Search Our Website&hellip;')? '' : this.value ;">
 <input type="submit" id="sf_submit" value="submit">
 </fieldset>
 </form>
 <nav>
 <ul>
 <li><a href="#">Text Link</a></li>
 <li><a href="#">Text Link</a></li>
 <li><a href="#">Text Link</a></li>
 <li><a href="#">Text Link</a></li>
 <li class="last"><a href="#">Text Link</a></li>
 </ul>
 </nav>
 </header>
</div>
<!-- content -->
<div class="wrapper row2">
 <div id="co

In [22]:
test_image = img_to_array(load_img('images/86.jpg', target_size=(299, 299)))
test_image = np.array(test_image, dtype=float)
test_image = preprocess_input(test_image)
test_features = IR2.predict(np.array([test_image]))
generate_desc(model, tokenizer, np.array(test_features), 100)

 <!DOCTYPE html>
<html lang="en" dir="ltr">
<head>
<title>Basic 86</title>
<meta charset="iso-8859-1">
<link rel="stylesheet" href="styles/layout.css" type="text/css">
<!--[if lt IE 9]><script src="scripts/html5shiv.js"></script><![endif]-->
</head>
<body>
<div class="wrapper row1">
 <header id="header" class="clear">
 <div id="hgroup">
 <h1><a href="#">Basic 86</a></h1>
 <h2>Free HTML5 Website Template</h2>
 </div>
 <nav>
 <ul>
 <li><a href="#">Text Link</a></li>
 <li><a href="#">Text Link</a></li>
 <li><a href="#">Text Link</a></li>
 <li><a href="#">Text Link</a></li>
 <li class="last"><a href="#">Text Link</a></li>
 </ul>
 </nav>
 </header>
</div>
<!-- content -->
<div class="wrapper row2">
 <div id="container" class="clear">
 <!-- Slider -->
 <section id="slider" class="clear">
 <figure><img src="images/demo/630x300.gif" alt="">
 <figcaption>
 <h2>Eu justo augue estas</h2>
 <p>Nullamlacus dui ipsum conseque loborttis non euisque morbi penas dapibulum orna. Urnaultrices quis curabit